In [1]:
import pandas as pd
from io import StringIO

In [2]:
from config import raw_path, processed_path
from utils import count_na, translate_to_english

C:\Users\pedro\AppData\Roaming\Python\Python39\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [3]:
valid_lines = []

with open(raw_path, 'r', encoding='utf-8') as file:
    for i, line in enumerate(file, start=1):
        try:
            pd.read_csv(StringIO(line), header=None)
            valid_lines.append(line)
        except pd.errors.ParserError:
            print(f"Linha {i} contém um erro de análise e será ignorada.")


df = pd.read_csv(StringIO(''.join(valid_lines)))

df.info()


Linha 29 contém um erro de análise e será ignorada.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           101 non-null    int64  
 1   Nome                         101 non-null    object 
 2   Idade                        101 non-null    int64  
 3   Gênero                       101 non-null    object 
 4   Localização                  101 non-null    object 
 5   Estado Civil                 101 non-null    object 
 6   Número de Dependentes        101 non-null    int64  
 7   Data de Início do Contrato   101 non-null    object 
 8   Data de Término do Contrato  4 non-null      object 
 9   Valor Mensal do Contrato     101 non-null    float64
 10  Tipo de Serviço              101 non-null    object 
 11  Duração Média das Chamadas   61 non-null     float64
 12  Volume de Dados           

In [4]:
df.head(3)

,ID,Nome,Idade,Gênero,Localização,Estado Civil,Número de Dependentes,Data de Início do Contrato,Data de Término do Contrato,Valor Mensal do Contrato,Tipo de Serviço,Duração Média das Chamadas,Volume de Dados,Número de Reclamações,Comentários
0,1,João Silva,35,Masculino,São Paulo/SP/São Paulo,Casado,2,2018-05-10,NaN,79.99,Telefonia Móvel,4.5,2.3 GB,2,O serviço de internet tem sido instável. Estou...
1,2,Maria Santos,28,Feminino,Rio de Janeiro/RJ/Rio de Janeiro,Solteiro,0,2019-03-15,NaN,109.99,Internet,NaN,-,0,Estou satisfeita com o serviço. A velocidade d...
2,3,Carlos Oliveira,42,Masculino,Belo Horizonte/MG/Minas Gerais,Casado,1,2020-01-20,2021-06-30,139.99,Internet,NaN,-,1,Tive problemas com a fatura deste mês. O valor...


In [5]:
count_na(df)

ID                              0
Nome                            0
Idade                           0
Gênero                          0
Localização                     0
Estado Civil                    0
Número de Dependentes           0
Data de Início do Contrato      0
Data de Término do Contrato    97
Valor Mensal do Contrato        0
Tipo de Serviço                 0
Duração Média das Chamadas     40
Volume de Dados                15
Número de Reclamações           0
Comentários                    33
dtype: int64

In [6]:
df['Comentários'] = df['Comentários'].fillna(df['Número de Reclamações'])


In [7]:
df['churn'] = df['Data de Término do Contrato'].notna().astype(int)

In [8]:
df[['cidade', 'estado_sigla', 'estado']] = df['Localização'].str.split('/', expand=True)

In [9]:
df['Duração Média das Chamadas'] = pd.to_numeric(df['Duração Média das Chamadas'], errors='coerce')
df['Número de Reclamações'] = pd.to_numeric(df['Número de Reclamações'], errors='coerce')

In [10]:
df['Volume de Dados'] = df['Volume de Dados'].str.extract('([\d.]+)', expand=False).astype(float)

In [11]:
df['Data de Início do Contrato'] = pd.to_datetime(df['Data de Início do Contrato'])
df['Data de Término do Contrato'] = pd.to_datetime(df['Data de Término do Contrato'])
df['duracao_contrato'] = pd.to_timedelta(df['Data de Término do Contrato'] - df['Data de Início do Contrato']).dt.days

In [12]:
df['Comentários'] = df['Comentários'].astype(str)
df['Tipo de Serviço'] = df['Tipo de Serviço'].astype(str)
df['estado_sigla'] = df['estado_sigla'].astype(str)
df['cidade'] = df['cidade'].astype(str)

df['Gênero'] = df['Gênero'].astype(str)
df['Estado Civil'] = df['Estado Civil'].astype(str)

df['cidade'] = df['cidade'].astype(str)
df['estado_sigla'] = df['estado_sigla'].astype(str)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID                           101 non-null    int64         
 1   Nome                         101 non-null    object        
 2   Idade                        101 non-null    int64         
 3   Gênero                       101 non-null    object        
 4   Localização                  101 non-null    object        
 5   Estado Civil                 101 non-null    object        
 6   Número de Dependentes        101 non-null    int64         
 7   Data de Início do Contrato   101 non-null    datetime64[ns]
 8   Data de Término do Contrato  4 non-null      datetime64[ns]
 9   Valor Mensal do Contrato     101 non-null    float64       
 10  Tipo de Serviço              101 non-null    object        
 11  Duração Média das Chamadas   61 non-null     

In [14]:
del(df['ID'])
del(df['Nome'])
del(df['Localização'])
del(df['estado'])

In [15]:
#df['comentarios_eng'] = df['Comentários'].apply(translate_to_english)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Idade                        101 non-null    int64         
 1   Gênero                       101 non-null    object        
 2   Estado Civil                 101 non-null    object        
 3   Número de Dependentes        101 non-null    int64         
 4   Data de Início do Contrato   101 non-null    datetime64[ns]
 5   Data de Término do Contrato  4 non-null      datetime64[ns]
 6   Valor Mensal do Contrato     101 non-null    float64       
 7   Tipo de Serviço              101 non-null    object        
 8   Duração Média das Chamadas   61 non-null     float64       
 9   Volume de Dados              13 non-null     float64       
 10  Número de Reclamações        53 non-null     float64       
 11  Comentários                  101 non-null    

In [21]:
from googletrans import Translator
translator = Translator()
# translator.translate('안녕하세요.')
# translator.translate('안녕하세요.', dest='ja')
#translator.translate('veritas lux mea', src='la')
# <Translated src=la dest=en text=The truth is my light pronunciation=The truth is my light>

In [36]:
import translators as ts

# def translate(comment):
#     return ts.google(comment, to_language='eng', sleep_seconds=2)

def translate(comment):
    return ts.translate_text(comment, translator='bing', to_language = 'en', sleep_seconds=1)

# print(ts.translators_pool)
# print(ts.translate_text(df['Comentários'][0], translator='bing', to_language = 'en', sleep_seconds=1))

In [37]:
df['comentarios_eng'] = df['Comentários'].apply(translate)

In [41]:
print(df['comentarios_eng'])

0      Internet service has been unstable. I'm having...
1      I am satisfied with the service. The internet ...
2      I had problems with this month's invoice. The ...
3      No complaints so far. The service has met my e...
4      The sports channel is not working properly. Th...
                             ...                        
96     I am extremely disappointed with this company'...
97     This company's internet is a real nightmare. T...
98     This company's cable TV service is a total dis...
99     This company's landline telephony is a disgrac...
100    This company's mobile phone is a complete disa...
Name: comentarios_eng, Length: 101, dtype: object


In [42]:
df.head()

,Idade,Gênero,Estado Civil,Número de Dependentes,Data de Início do Contrato,Data de Término do Contrato,Valor Mensal do Contrato,Tipo de Serviço,Duração Média das Chamadas,Volume de Dados,Número de Reclamações,Comentários,churn,cidade,estado_sigla,duracao_contrato,comentarios_eng
0,35,Masculino,Casado,2,2018-05-10,NaT,79.99,Telefonia Móvel,4.5,2.3,2.0,O serviço de internet tem sido instável. Estou...,0,São Paulo,SP,NaN,Internet service has been unstable. I'm having...
1,28,Feminino,Solteiro,0,2019-03-15,NaT,109.99,Internet,NaN,NaN,0.0,Estou satisfeita com o serviço. A velocidade d...,0,Rio de Janeiro,RJ,NaN,I am satisfied with the service. The internet ...
2,42,Masculino,Casado,1,2020-01-20,2021-06-30,139.99,Internet,NaN,NaN,1.0,Tive problemas com a fatura deste mês. O valor...,1,Belo Horizonte,MG,527.0,I had problems with this month's invoice. The ...
3,55,Feminino,Viúvo,0,2017-09-01,NaT,159.99,Telefonia Fixa,6.2,NaN,NaN,Sem reclamações até o momento. O serviço tem a...,0,Porto Alegre,RS,NaN,No complaints so far. The service has met my e...
4,41,Masculino,Divorciado,3,2019-11-12,2022-01-05,99.99,TV a Cabo,NaN,NaN,0.0,O canal de esportes não está funcionando corre...,1,Brasília,DF,785.0,The sports channel is not working properly. Th...


In [30]:
print(translate(df['Comentários'][0]))

AttributeError: module 'translators' has no attribute 'google'

In [39]:
df.to_parquet(processed_path)